# Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as  tf
import numpy as np
import itertools
from keras.optimizers import Adam
from keras.models import clone_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from functools import partial
import itertools
import random


2025-03-23 13:55:48.379197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 13:55:48.385861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742748948.393119  173647 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742748948.395454  173647 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 13:55:48.403771: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam

In [3]:
from tqdm import tqdm
import cv2

# Definitions

In [4]:
pwd

'/home/nicolas/Documentos/UTN/INA/giar_ina_dev/notebooks/Autoencoder'

In [5]:
# base_path = "/content/drive/My Drive/Investigacion/UTN/GIAR/Dataset/cropped_cells"
# save_path = "/content/drive/My Drive/Investigacion/UTN/GIAR/Results/Autoencoder/encoder_SSIM_MAE_Bparams.keras"

base_path = "../../output/cropped_cells_full_v3"
save_path = "../../models/encoder_SSIM_MAE_Bparams_vN.keras"


SHAPE = (128,128,1)
batch_size = 300
validation_split = 0.2

In [6]:
ON_RAM = False

# Load dataset

In [7]:

if ON_RAM:

    images = np.zeros((len(os.listdir(base_path)), SHAPE[0], SHAPE[1]))
    for idx, file in tqdm(enumerate(os.listdir(base_path)), total=len(os.listdir(base_path))):
        image = cv2.imread(os.path.join(base_path, file), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SHAPE[0], SHAPE[1]))
        image = np.array(image) #

        images[idx, : ,:] = image


    from sklearn.model_selection import train_test_split
    x_train, x_test = train_test_split(images, test_size=validation_split, random_state=42)
    x_train.shape

else:

    image_paths = list(tf.io.gfile.glob(base_path + '/*.jpg'))

    # Train / val split
    val_size = int(0.2 * len(image_paths))  # 20% para validación
    train_paths = image_paths[:-val_size]
    val_paths = image_paths[-val_size:]

    train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
    val_dataset = tf.data.Dataset.from_tensor_slices(val_paths)

    # Load & augmentation
    def load_image(path):

      image = tf.io.read_file(path)
      image = tf.image.decode_png(image, channels=1)  # Escala de grises
      image = tf.image.resize(image, (SHAPE[0], SHAPE[1]))
      # Augmentación

      image = tf.image.random_flip_left_right(image)
      image = tf.image.random_flip_up_down(image)
      image = tf.image.random_brightness(image, max_delta=0.1)
      image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
      scales = tf.random.uniform([], 0.8, 1.0)
      crop_size = tf.cast(scales * SHAPE[0], tf.int32)
      image = tf.image.random_crop(image, size=[crop_size, crop_size, 1])
      image = tf.image.resize(image, (SHAPE[0], SHAPE[1]))
      # Normalize [0, 1]
      image = tf.cast(image, tf.float32) / 255.0
      return image,image

    # Train dataset
    train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
    train_dataset = train_dataset.map(lambda x: load_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_dataset = train_dataset.shuffle(buffer_size=10000)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    train_dataset = train_dataset.repeat()

    # Val dataset
    validation_dataset = val_dataset.map(lambda x: load_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size)
    validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.repeat()



I0000 00:00:1742748949.458793  173647 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20835 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
len(train_paths)

42880

# Weigthed Loss: SSIM + MAE

In [9]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, y, z, name="custom_loss"):
        super().__init__(name=name)
        # Weights
        self.y = y
        self.z = z

        #MAE
        self.mae_loss_fn = tf.keras.losses.MeanAbsoluteError()

    # SSIM loss
    def ssim_loss(self, y_true, y_pred):

        ssim = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
        return ssim

    def call(self, y_true, y_pred):

        mae_loss = self.mae_loss_fn(y_true, y_pred)
        ssim = self.ssim_loss(y_true, y_pred)


        return  self.y * mae_loss + self.z * ssim

# AutoEncoder

In [10]:
kernel_size = (4,4)
filter = 32

# Encoder
encoder_input = layers.Input(shape=(128, 128, 1))

x = encoder_input

x = layers.Rescaling(1.0 / 255.0)(x)


x = layers.Conv2D(filter, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//2, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//4, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//8, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//16, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Flatten()(x)

encoder_output = x


encoder = Model(encoder_input, encoder_output)

# Decoder

decoder_input = layers.Input(shape=[encoder.output_shape[-1]])

x = decoder_input

x = layers.Reshape((2*filter//16, 2*filter//16, filter//16))(x)


x = layers.Conv2D(filter//16, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter//8, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter//4, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter//2, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(1, kernel_size, activation='sigmoid', padding='same')(x)

#x = layers.Rescaling(255.0)(x)

decoder_output = x

decoder = Model(decoder_input, decoder_output)

# Build the autoencoder model
# autoencoder = Model(encoder_input, decoder_output)
autoencoder = Sequential([
    encoder,
    decoder
])

In [11]:
encoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 16)     │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 8)      │         2,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 4)      │           516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 4)      │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 2)        │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 2)        │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 4, 4, 2)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,702 (45.71 KB)

 Trainable params: 11,578 (45.23 KB)

 Non-trainable params: 124 (496.00 B)

In [12]:
decoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 4, 4, 2)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 2)        │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 4, 2)        │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 8, 8, 2)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 4)        │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 8, 8, 4)        │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 16, 16, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 8)      │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 16, 16, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 32, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 16)     │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 32, 32, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 32)     │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 1)    │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,767 (45.96 KB)

 Trainable params: 11,643 (45.48 KB)

 Non-trainable params: 124 (496.00 B)

In [13]:
# Summary of the model
autoencoder.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional (Functional)         │ (None, 32)             │        11,702 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 128, 128, 1)    │        11,767 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,469 (91.68 KB)

 Trainable params: 23,221 (90.71 KB)

 Non-trainable params: 248 (992.00 B)

# Compile & fit - Random search

In [14]:
# Weight range from 0.2 to 1
y_values = np.arange(0.2, 1.2, 0.2)
z_values = np.arange(0.2, 1.2, 0.2)

# Combinations of weights
param_combinations = list(itertools.product(y_values,z_values))

random.seed(34)
random.shuffle(param_combinations)

# Normalize weights ---> y + z = 1
def normalize_weights(weights):
    total_sum = sum(weights)
    return tuple(weight / total_sum for weight in weights)

param_combinations = [normalize_weights(weights) for weights in param_combinations]
param_combinations =set(param_combinations) #delete duplicate
print("Total combinations: ", len(param_combinations))
for i,combination in enumerate(param_combinations):
    print(f"Combination {i+1}: ", tuple(map(float, combination)))

Total combinations:  19
Combination 1:  (0.25, 0.7500000000000001)
Combination 2:  (0.37500000000000006, 0.625)
Combination 3:  (0.3333333333333333, 0.6666666666666666)
Combination 4:  (0.7142857142857143, 0.28571428571428575)
Combination 5:  (0.4285714285714286, 0.5714285714285714)
Combination 6:  (0.4444444444444445, 0.5555555555555556)
Combination 7:  (0.5555555555555556, 0.4444444444444445)
Combination 8:  (0.7500000000000001, 0.25)
Combination 9:  (0.5, 0.5)
Combination 10:  (0.28571428571428575, 0.7142857142857143)
Combination 11:  (0.5714285714285714, 0.4285714285714286)
Combination 12:  (0.625, 0.37500000000000006)
Combination 13:  (0.8333333333333334, 0.16666666666666669)
Combination 14:  (0.6000000000000001, 0.4)
Combination 15:  (0.6666666666666666, 0.3333333333333333)
Combination 16:  (0.2, 0.8)
Combination 17:  (0.4, 0.6000000000000001)
Combination 18:  (0.8, 0.2)
Combination 19:  (0.16666666666666669, 0.8333333333333334)


In [15]:
def evaluate_model(autoencoder, train_gen, val_gen, params, steps_per_epoch, val_steps):
    y, z = [np.round(arr, 2) for arr in params]

    loss_fn = CustomLoss(y=y, z=z)
    model = clone_model(autoencoder)
    model.compile(
        loss=loss_fn,
        optimizer=Adam(learning_rate=1e-3)
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=0
    )

    history=model.fit(
        train_gen,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        validation_data=val_gen,
        validation_steps=val_steps,
        callbacks=[early_stop],
        verbose=1
        )

    val_loss = min(history.history['val_loss'])
    return val_loss


best_params = None
best_loss = float('inf')


steps_per_epoch = int(len(train_paths) // batch_size)
steps_per_epoch_val = int(len(val_paths) // batch_size)


# Evaluate Autoencoder for all the combinations of weights
for params in param_combinations:
    try:
        val_loss = evaluate_model(autoencoder, train_dataset, validation_dataset, params, steps_per_epoch, steps_per_epoch_val)
        print(f"Params {params} -> Val Loss: {val_loss}")

        if val_loss < best_loss:
            best_loss = val_loss
            best_params = params
    except Exception as e:
        print(f"{e}")

# Resultados finales
print(f"Best Params: {best_params} -> Best Val Loss: {best_loss}")

Epoch 1/20


I0000 00:00:1742748953.953459  173809 service.cc:148] XLA service 0x7ed8640095a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742748953.953478  173809 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-03-23 13:55:54.029222: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1742748954.174983  173809 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert/Assert
W0000 00:00:1742748954.175274  173809 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_1/Assert
W0000 00:00:1742748954.175477  173809 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_2/Assert
W0000 00:00:1742748954.175612  173809 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_3/Assert
I0000 00:00:1

  2/142 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - loss: 0.7430  

I0000 00:00:1742748967.207400  173809 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.4571

...

142/142 ━━━━━━━━━━━━━━━━━━━━ 23s 75ms/step - loss: 0.3634 - val_loss: 0.4234
Epoch 3/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3548 - val_loss: 0.4178
Epoch 4/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3486 - val_loss: 0.4159
Epoch 5/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.3449 - val_loss: 0.4161
Epoch 6/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3430 - val_loss: 0.5497
Epoch 7/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3417 - val_loss: 0.8281
Epoch 8/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3393 - val_loss: 0.6894
Epoch 9/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3372 - val_loss: 0.3865
Epoch 10/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3370 - val_loss: 0.3371
Epoch 11/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3345 - val_loss: 0.4447
Epoch 12/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.3328 - val_loss: 0.4593
Epoch 13/20
142/142 ━━━━━━━━━━━━━━━━━━━━

W0000 00:00:1742749146.139260  173812 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert/Assert
W0000 00:00:1742749146.139621  173812 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_1/Assert
W0000 00:00:1742749146.139835  173812 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_2/Assert
W0000 00:00:1742749146.139970  173812 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_3/Assert


142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.4018

W0000 00:00:1742749158.597934  173811 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert/Assert
W0000 00:00:1742749158.598306  173811 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_1/Assert
W0000 00:00:1742749158.598506  173811 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_2/Assert
W0000 00:00:1742749158.598637  173811 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SSIM/Assert_3/Assert


142/142 ━━━━━━━━━━━━━━━━━━━━ 16s 81ms/step - loss: 0.4014 - val_loss: 0.3981
Epoch 2/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 13s 76ms/step - loss: 0.3171 - val_loss: 0.4554
Epoch 3/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 78ms/step - loss: 0.3074 - val_loss: 0.4833
Epoch 4/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.3017 - val_loss: 0.3820
Epoch 5/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.2970 - val_loss: 0.3412
Epoch 6/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.2964 - val_loss: 0.3072
Epoch 7/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.2945 - val_loss: 0.4953
Epoch 8/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 78ms/step - loss: 0.2933 - val_loss: 0.3454
Epoch 9/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.2933 - val_loss: 0.3037
Epoch 10/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.2927 - val_loss: 0.4374
Epoch 11/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - loss: 0.2908 - val_loss: 0.4723
Epoch 12/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 